In [49]:
import pandas as pd
import datetime as dt
import numpy as np
import ast
import re
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import matplotlib.pyplot as plt

FONCTIONS

In [2]:
def str_to_list(element):
    element = element.replace('"', "").replace("[", '').replace("]", '').replace("'", '')
    element = element.split(', ')
    return element

def en_set(element):
    element = set(element)
    return element

def list_propre(element):
    element = str(element)
    element = element.replace('"', "").replace("[", '').replace("]", '').replace("'", '')
    return element

def str_to_set(element):
    element = element.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(", ", ',')
    element = set(element.split(','))
    return element

def str_to_set_to_list(element):
    element = element.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(", ", ',')
    element = list(set(element.split(',')))
    return element

def debut2(titre):
    titre = titre.lower().replace("'", '').replace('(', '').replace(')', '').replace('[', '').replace(']', '').replace('{', '').replace('}', '').replace('/', '').replace(' ', '').replace('.', '').replace(',', '').replace('&', '').replace('?', '').replace('!', '')
    titre_final = ''
    if ':' in titre:
        for n in range(len(titre)):
            if titre[n] != ':':
                if titre[n].isalpha() or titre[n].isnumeric():
                    titre_final += titre[n]   
            else:
                break           

    return titre_final

def name(liste_films, df):

    df = df.drop(['ordering', 'category', 'job', 'characters'], axis = 1)
    df['nconst'] = df['nconst'] + ', '
    df = df.groupby('tconst').sum()
    df = df.reset_index()

    df = pd.merge(liste_films,
                  df,
                  left_on = 'film_id_out_KNN',
                  right_on = 'tconst',
                  how = 'left')

    df = df.drop('film_id_out_KNN', axis = 1)
    df['nconst'] = df['nconst'].astype(str)
    df['nconst'] = df['nconst'].apply(lambda x : x.split(','))
    df = df.explode('nconst')
    df['tconst'] = df['tconst'] + ','
    df = df.groupby('nconst').sum()
    df = df.reset_index()
    df = df[~(df['nconst']== ' ')]
    df['nconst'] = df['nconst'].astype(str)
    df['tconst'] = df['tconst'].astype(str)
    df['nconst'] = df['nconst'].apply(lambda x : x.replace(' ', ''))
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', ''))

    return df

def acteur_name(df):

    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + df['tconst_y']
    df = df[['nconst', 'primaryName', 'tconst']]
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", ''))
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df = df.rename({'tconst': 'knownForTitles'}, axis = 1)
    df = df.rename({'nconst': 'acteur'}, axis = 1)

    return df

def realisateur_name(df):
    
    df = df.groupby('nconst').sum()
    df = df.reset_index()
    df = pd.merge(df,
                  df_title_crew,
                  on = 'nconst',
                  how = 'left')

    df = df[~(df['nconst'] == 'nan')]
    df['tconst_x'] = df['tconst_x'].astype(str)
    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + ',' + df['tconst_y']
    df = df.drop(['tconst_y', 'tconst_x'], axis = 1)
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ','))
    df['tconst'] = df['tconst'].astype(str)
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df['tconst'] = df['tconst'].apply(lambda x : set(x))
    df['tconst'] = df['tconst'].apply(lambda x : list(x))

    df = pd.merge(df,
                  df_name_basics,
                  on = 'nconst',
                  how = 'left')

    df['tconst_x'] = df['tconst_x'].astype(str)
    df['tconst_y'] = df['tconst_y'].astype(str)
    df['tconst'] = df['tconst_x'] + ',' + df['tconst_y']
    df = df.drop(['tconst_y', 'tconst_x'], axis = 1)
    df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", '').replace(",,", ','))
    df['tconst'] = df['tconst'].astype(str)
    df['tconst'] = df['tconst'].apply(lambda x : x.split(','))
    df['tconst'] = df['tconst'].apply(lambda x : set(x))
    df['tconst'] = df['tconst'].apply(lambda x : list(x))

    df = df.rename({'tconst': 'knownForTitles'}, axis = 1)
    df = df.rename({'nconst': 'realisateurs'}, axis = 1)
    df = df.explode('knownForTitles')
    df = df[~(df['knownForTitles'] == '')]

    return df

def debut(titre):
    titre = titre.lower().replace(' ', '')
    titre_final = ''
    for n in range(len(titre)):
        if titre[n].isalpha():
            titre_final += titre[n]
    return titre_final[0:20]

def len_pandas(x, colonne):
    x_escaped = re.escape(x)
    return df_films[df_films[f'{colonne}'].fillna('').str.contains(x_escaped, na=False, regex=True)].shape[0]

def liste_recurrence(df_films, colonne):

    df_films[colonne] = df_films[colonne].astype(str)
    df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
    df_films[f'{colonne}'] = df_films[f'{colonne}2'].apply(lambda x : x.split(','))

    liste= []

    for genres in df_films[colonne] :
        for element in genres:
            liste.append(element)

    set_liste = set(liste)

    if 'nan' in set_liste:
        set_liste.remove('nan')
    if "" in set_liste:
        set_liste.remove("")

    dico_production_companies_name_out_KNN = {}

    for element in set_liste:
        if colonne == 'production_companies_name_out_KNN':
            if liste.count(element) > apparition_production:
                if len(element) > len_production:
                    dico_production_companies_name_out_KNN.update({element : liste.count(element)})
        else:
            dico_production_companies_name_out_KNN.update({element : liste.count(element)})

    sorted_set = dict(sorted(dico_production_companies_name_out_KNN.items(), key=lambda item:item[1], reverse = True))
    
    limite_realisateurs = 100
    limite_acteur = 200
    limite_production_companies_name = 100
    limite_genre = len(set_liste)

    if colonne == 'acteur_out_KNN':
        limite = limite_acteur
    elif colonne == 'realisateurs_out_KNN':
        limite = limite_realisateurs
    elif colonne == 'production_companies_name_out_KNN':
        limite = limite_production_companies_name
    elif colonne == 'genre_out_KNN':
        limite = limite_genre

    liste_totale = list(sorted_set.keys())

    if len(liste_totale) > limite_liste:
        liste_totale = liste_totale[:limite_liste]

    
    liste_limitee = liste_totale[:limite]

    return liste_totale, liste_limitee

def classement_element(df_films, colonne, liste_totale, dico_total_final):

  if colonne in ['acteur_out_KNN', 'realisateurs_out_KNN']:
    df_films[f'trie_{colonne}'] = ''
    df_films[f'name_trie_{colonne}'] = ''

    for element in liste_totale:
      df_films[f'trie_{colonne}'] = df_films.apply(lambda row : (row[f'trie_{colonne}'] + f', {element}') if element in row[colonne] else row[f'trie_{colonne}'], axis = 1)
      df_films[f'name_trie_{colonne}'] = df_films.apply(lambda row : (row[f'name_trie_{colonne}'] + f', {dico_total_final[element]}') if element in row[colonne] else row[f'name_trie_{colonne}'], axis = 1)
    
    df_films[f'trie_{colonne}'] = df_films[f'trie_{colonne}'].apply(lambda x : x[2:])
    df_films[f'name_trie_{colonne}'] = df_films[f'name_trie_{colonne}'].apply(lambda x : x[2:])

    df_films[f'trie_{colonne}'] = df_films[f'trie_{colonne}'].apply(lambda x : x.split(', '))
    df_films[f'name_trie_{colonne}'] = df_films[f'name_trie_{colonne}'].apply(lambda x : x.split(', '))

  else:
    df_films[f'trie_{colonne}'] = ''

    for element in liste_totale:
      df_films[f'trie_{colonne}'] = df_films.apply(lambda row : (row[f'trie_{colonne}'] + f', {element}') if element in row[colonne] else row[f'trie_{colonne}'], axis = 1)

    df_films[f'trie_{colonne}'] = df_films[f'trie_{colonne}'].apply(lambda x : x[2:])

    df_films[f'trie_{colonne}'] = df_films[f'trie_{colonne}'].apply(lambda x : x.split(', '))

def bool_colonnes(df_films, liste, colonne):

    for genre in liste:
        df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)

def transfo_col_bool(df_films):

    df_films_not_bool = df_films.select_dtypes(exclude = 'bool')
    df_films_bool = df_films.select_dtypes(include = 'bool')
    df_films_bool = df_films_bool.astype(str)
    df_films_bool = df_films_bool.replace('True', '1').replace('False', '0')
    df_films_bool = df_films_bool.astype(int)
    df_films = pd.concat([df_films_not_bool, df_films_bool], axis = 1)
    df_films_not_bool, df_films_bool = 0, 0

    return df_films

def colonne_debut_titre(df_films, colonne):

    liste_titres = []

    for n in range(len(df_films)):
        liste_titres.append(debut2(df_films[colonne].iloc[n]))

    dico_titres = {}
    set_titres = set(liste_titres)

    for element in set_titres:
        if len(element) > 4:
            if liste_titres.count(element) >=3:
                dico_titres.update({element : liste_titres.count(element)})

    if '' in set_titres:
        set_titres.remove('')
        
    sorted_dico_titres = dict(sorted(dico_titres.items(), key=lambda item:item[1], reverse = True))  

    df_films[f'debut{colonne}'] = df_films[colonne].apply(debut2)

    df_films[f'debut_critere_{colonne}'] = ''

    df_films[f'debut_critere_{colonne}'] = df_films.apply(lambda x : x[f'debut{colonne}'] if x[f'debut{colonne}'] in sorted_dico_titres.keys() else '', axis = 1)
    df_films[f'debut_critere_{colonne}'] = df_films[f'debut_critere_{colonne}'].apply(lambda x : x.split())

    liste_limitee_title = sorted_dico_titres.keys()

    return liste_limitee_title


def clean_list(element):

    for element2 in element:
        if element2 == '':
            element.remove(element2)
        if element2 == 'nan':
            element.remove(element2)

    return element
            
def list_colonnes(df_films, colonne):

    df_films[colonne] = df_films[colonne].astype(str)
    df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())
    df_films[f'{colonne}2'] = df_films[f'{colonne}2'].apply(lambda x : x.replace('nan', '') if 'nan' in x else x)
    df_films[f'{colonne}'] = df_films[f'{colonne}2'].apply(lambda x : x.split(','))
    df_films[f'{colonne}'] = df_films[f'{colonne}'].apply(lambda x : clean_list(x))

    # for n in range(len(df_films)):
    #     for element in df_films[colonne].iloc[n]:
              
    #         if element == "":
    #             df_films[colonne].iloc[n].remove(element)
    #         elif element == "nan":
    #             df_films[colonne].iloc[n].remove(element)

IMPORTS ET MERGE

In [3]:
df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])
for element in df_tmbd_full.columns:
    df_tmbd_full = df_tmbd_full.rename({element : f'{element}_tmdb'}, axis = 1)
    
df_tmbd_full = df_tmbd_full.drop(['budget_tmdb','revenue_tmdb'], axis = 1)

df_tmbd_full = df_tmbd_full.rename({'imdb_id_tmdb': 'film_id_tmbd',
                                    'original_language_tmdb': 'language1_tmdb',
                                    'original_title_tmdb' : 'title1_tmdb',
                                    'release_date_tmdb': 'year1_tmdb',
                                    'spoken_languages_tmdb' : 'language2_tmdb',
                                    'vote_average_tmdb' : 'vote_exact_tmdb',
                                    'production_companies_country_tmdb': 'country2_tmdb',
                                    'title_tmdb' : 'title2_tmdb'}, axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20590/3742720658.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tmbd_full = pd.read_csv('BD/tmdb_full.csv', na_values= ['\\N'])


In [4]:
df_title_ratings = pd.read_csv('BD/title.ratings.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])
for element in df_title_ratings.columns:
    df_title_ratings = df_title_ratings.rename({element : f'{element}_title_ratings'}, axis = 1)

In [5]:
df_title_ratings = df_title_ratings.rename({'tconst_title_ratings': 'film_id_title_ratings',
                                     'averageRating_title_ratings' : 'vote_exact_title_ratings',
                                    'numVotes_title_ratings': 'vote_count_title_ratings'}, axis = 1)

In [6]:
df_films = pd.merge(df_tmbd_full,
                    df_title_ratings,
                    how = 'left',
                    left_on = "film_id_tmbd",
                    right_on = 'film_id_title_ratings')

In [7]:
df_tmbd_full, df_title_ratings = 0,0

In [8]:
df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])
for element in df_title_basics.columns:
    df_title_basics = df_title_basics.rename({element : f'{element}_title_basics'}, axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20590/2103303214.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('BD/title.basics.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])


In [9]:
df_title_basics = df_title_basics.drop(['endYear_title_basics'], axis = 1)

df_title_basics = df_title_basics.rename({'tconst_title_basics': 'film_id_title_basics',
                                          'primaryTitle_title_basics': 'title1_title_basics',
                                          'originalTitle_title_basics': 'title2_title_basics',
                                          'startYear_title_basics' : 'year_title_basics',
                                          'runtimeMinutes_title_basics': 'runtime_title_basics',
                                          'genres_title_basics':'genre_title_basics'}, axis = 1)

In [10]:
df_films = pd.merge(df_films,
                    df_title_basics,
                    how = 'left',
                    left_on = "film_id_tmbd",
                    right_on = 'film_id_title_basics')

In [11]:
df_title_basics = 0

In [12]:
df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])
for element in df_title_akas.columns:
    df_title_akas = df_title_akas.rename({element : f'{element}_title_akas'}, axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20590/2837915362.py:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('BD/title.akas.tsv.gz', compression = 'gzip', sep = '\t', na_values= ['\\N'])


In [13]:
df_title_akas = df_title_akas.drop(['attributes_title_akas','ordering_title_akas', 'types_title_akas', 'isOriginalTitle_title_akas'], axis = 1)

df_title_akas = df_title_akas.rename({'titleId_title_akas': 'film_id_title_akas',
                                      'region_title_akas': 'country_title_akas',}, axis = 1)

In [14]:
df_films = pd.merge(df_films,
                    df_title_akas,
                    how = 'left',
                    left_on = "film_id_tmbd",
                    right_on = 'film_id_title_akas')

In [15]:
df_title_akas = 0

TRI ET NETTOYAGE

In [16]:
# ON NE GARDE QUE LES FILMS
 
df_films = df_films[df_films['titleType_title_basics'] == 'movie']

DROP DES DUPLICATES

In [21]:
df_films = df_films.drop_duplicates(subset = 'film_id_tmbd', keep = 'first')

POPULARITY

In [17]:
df_films = df_films.rename({'popularity_tmdb' : 'popularity_final'}, axis = 1)

TITLE

In [50]:
def titre(id):
    navigator = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1)'
    url_base_title = 'https://www.imdb.com/fr/title/'
    url_finale_title = f'{url_base_title}{id}'  

    html_title = requests.get(url_finale_title, headers={'User-Agent': navigator})
    html_title2 = html_title.content
    soup_title = BeautifulSoup(html_title2, 'html.parser')

    for balise_parent in soup_title.find_all('span', class_='hero__primary-text'):
        titre = balise_parent.get_text().strip()

    return titre

RUNTIME

In [18]:
travail = 'runtime'
colonnes = []
for element in df_films.columns:
    if travail in element:
        colonnes.append(element)

df_runtime = df_films[colonnes]

for element in df_runtime.columns:
    df_runtime[element] = df_runtime[element].astype(float)
    df_runtime[element] = df_runtime[element].apply(lambda x : np.nan if x == 0 else x)

df_films['runtime_final'] = df_runtime['runtime_tmdb'].fillna(df_runtime['runtime_title_basics'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/4216881176.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_runtime[element] = df_runtime[element].astype(float)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/4216881176.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_runtime[element] = df_runtime[element].apply(lambda x : np.nan if x == 0 else x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/4216881176.py:10: SettingWithCopyWarning: 
A valu

YEAR + DECENNIE

In [19]:
travail = 'year'

colonnes = []
for element in df_films.columns:
    if travail in element:
        colonnes.append(element)

df_year = df_films[colonnes]

df_year['year1_tmdb'] = pd.to_datetime(df_year['year1_tmdb']).dt.year

for element in df_year.columns:
    df_year[element] = df_year[element].apply(lambda x : round(float(x), 0))
    df_year[element] = df_year[element].apply(lambda x : np.nan if x == 0 else x)


df_films['year_final'] = df_year['year1_tmdb'].fillna(df_year['year_title_basics'])
df_films['year_final'] = df_films['year_final'].fillna(0)
df_films['year_final'] = df_films['year_final'].apply(lambda x : float(x))
df_films['year_final'] = df_films['year_final'].apply(lambda x : int(x))

df_films['Decennie'] = ''
df_films['year_final'] = df_films['year_final'].astype(str)
df_films['Decennie'] = df_films['year_final'] .str[:3] + "0"
df_films['year_final']  = pd.to_numeric(df_films['year_final'] )
df_films['Decennie'] = pd.to_numeric(df_films['Decennie'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/1732751102.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year['year1_tmdb'] = pd.to_datetime(df_year['year1_tmdb']).dt.year
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/1732751102.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_year[element] = df_year[element].apply(lambda x : round(float(x), 0))
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/1732751102.py:14: SettingWithCopyWarning: 
A v

VOTE EXACT + VOTE ARRONDI

In [20]:
travail = 'vote_exact'
colonnes = []

for element in df_films.columns:
    if travail in element:
        colonnes.append(element)

df_vote_exact = df_films[colonnes]

for element in df_vote_exact.columns:
    df_vote_exact[element] = df_vote_exact[element].astype(float)
    df_vote_exact[element] = df_vote_exact[element].apply(lambda x : np.nan if x == 0 else x)

df_films['vote_exact_final'] = df_vote_exact['vote_exact_tmdb'].fillna(df_vote_exact['vote_exact_title_ratings'])
df_films['vote_exact_final'] = df_films['vote_exact_final'].fillna(0)
df_films['vote_exact_final'] = df_films['vote_exact_final'].astype(float)
df_films['vote_arrondi_final'] = df_films['vote_exact_tmdb'].apply(lambda x : int(x))

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3276840401.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vote_exact[element] = df_vote_exact[element].astype(float)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3276840401.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vote_exact[element] = df_vote_exact[element].apply(lambda x : np.nan if x == 0 else x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3276840401.py:11: SettingWithCopyWarn

VOTE COUNT

In [21]:
travail = 'vote_count'

colonnes = []

for element in df_films.columns:
    if travail in element:
        colonnes.append(element)

df_vote_count = df_films[colonnes]

for element in df_vote_count.columns:
    df_vote_count[element] = df_vote_count[element].astype(float)
    df_vote_count[element] = df_vote_count[element].apply(lambda x : np.nan if x == 0 else x)

df_films['vote_count_final'] = df_vote_count['vote_count_tmdb'].fillna(df_vote_count['vote_count_title_ratings'])

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/4059487617.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vote_count[element] = df_vote_count[element].astype(float)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/4059487617.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vote_count[element] = df_vote_count[element].apply(lambda x : np.nan if x == 0 else x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/4059487617.py:12: SettingWithCopyWarn

PRODUCTION

In [22]:
df_films = df_films.rename({'production_companies_name_tmdb': 'production_companies_name_final'}, axis = 1)

GENRE

In [23]:
travail = 'genre'
colonnes = []

for element in df_films.columns:
    if travail in element:
        colonnes.append(element)

df_genre = df_films[colonnes]

for element in colonnes:
    df_genre[element] = df_genre[element].astype(str)
    df_genre[element] = df_genre[element].apply(lambda x : list_propre(x))

df_films['genre_final'] = ''
df_films['genre_final']  = df_genre['genres_tmdb']  + ',' + df_genre['genre_title_basics']
df_films['genre_final'] = df_films['genre_final'].apply(lambda x : str_to_set_to_list(x))

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/1761951680.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre[element] = df_genre[element].astype(str)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/1761951680.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_genre[element] = df_genre[element].apply(lambda x : list_propre(x))
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/1761951680.py:11: SettingWithCopyWarning: 
A value is trying to be s

LANGUAGE + COUNTRY

In [25]:
travail = ['language', 'country']
colonnes = []
for element in df_films.columns:
    for element2 in travail:
        if element2 in element:
            colonnes.append(element)

df_country = df_films[colonnes]

for element in df_country.columns:
    df_country[element] = df_country[element].astype(str)
    df_country[element] = df_country[element].apply(lambda x : list_propre(x.lower()))
    df_country[element] = df_country[element].apply(lambda x : np.nan if x == 'nan' else x)

df_country['language_total'] = df_country['language1_tmdb'] + ',' + df_country['language2_tmdb'] + ',' + df_country['country2_tmdb'] + ',' + df_country['country_title_akas'] + ',' + df_country['language_title_akas']

df_country['language_total'] = df_country['language_total'].astype(str)
df_films['language_total_FR_ou_US'] = ''
df_films['language_total_FR_ou_US'] = df_country.apply(lambda x : 'True' if 'fr' in x['language_total'] or 'en' in x['language_total'] or 'us' in x['language_total'] or x['language_total'] == 'nan' else 'False', axis = 1)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/5757658.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country[element] = df_country[element].astype(str)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/5757658.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_country[element] = df_country[element].apply(lambda x : list_propre(x.lower()))
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/5757658.py:13: SettingWithCopyWarning: 
A value is trying 

PRODUCTION COUNTRY

In [26]:
df_films['production_countries_tmdb'] = df_films['production_countries_tmdb'].apply(lambda x : str_to_set_to_list(x))

In [27]:
df_films['prod_FR'] = df_films.apply(lambda row : 1 if 'FR' in row['production_countries_tmdb'] else 0, axis = 1)
df_films['prod_US'] = df_films.apply(lambda row : 1 if 'US' in row['production_countries_tmdb'] else 0, axis = 1)

PRODUCTION COMPANY

In [28]:
df_films['production_companies_name_final'] = df_films['production_companies_name_final'].apply(lambda x : str_to_set_to_list(x))

ON GARDE SEULEMENT LES COLONNES SOUHAITEES

In [29]:
df_films = df_films[['adult_tmdb', 'backdrop_path_tmdb', 'homepage_tmdb',
                     'film_id_tmbd', 'overview_tmdb', 'popularity_final', 'poster_path_tmdb',
                     'status_tmdb', 'tagline_tmdb', 'video_tmdb', 'production_companies_name_final', 'runtime_final',
                     'Decennie', 'year_final', 'vote_exact_final', 'title_final',
                     'vote_arrondi_final', 'genre_final', 'production_countries_tmdb', 'language_total_FR_ou_US',
                     'prod_US', 'prod_FR', 'vote_count_final']]

In [30]:
df_films = df_films.rename({'adult_tmdb' : 'adult_out_KNN',
                            'backdrop_path_tmdb' : 'backdrop_path_out_KNN',
                            'homepage_tmdb' : 'homepage_out_KNN',
                            'film_id_tmbd' : 'film_id_out_KNN',
                            'overview_tmdb' : 'overview_out_KNN',
                            'poster_path_tmdb' : 'poster_path_out_KNN',
                            'status_tmdb' : 'status_out_KNN',
                            'tagline_tmdb' : 'tagline_tmdb_out_KNN',
                            'video_tmdb' : 'video_tmdb_out_KNN',
                            'production_countries_tmdb' : 'production_countries_out_KNN',
                            'language_total_FR_ou_US' : 'language_total_FR_ou_US_out_KNN',
                            'title_final': 'title_out_KNN',
                            'genre_final' : 'genre_out_KNN',
                            'production_companies_name_final':'production_companies_name_out_KNN'}, axis = 1)

DROP DES DUPLICATES

In [31]:
df_films = df_films.drop_duplicates(subset = 'film_id_out_KNN', keep = 'first')

TRI DES TITRES PAR LE CYRILLIQUE

In [32]:
df_films = df_films[~df_films['title_final_out_KNN'].str.contains(r'[^\x00-\x7F]+', regex=True)]

CHOIX DES CRITERES

In [33]:
# SUR DF
df_films = df_films[df_films['popularity_final'] >= 0]
df_films = df_films[df_films['Decennie'] >= 1980]
df_films = df_films[df_films['vote_exact_final'] > 5]
df_films = df_films[df_films['runtime_final'] > 60]
df_films = df_films[df_films['status_out_KNN'] == 'Released']
df_films = df_films[df_films['language_total_FR_ou_US_out_KNN'] == 'True']


# POUR FONCTIONS
limite_liste = 500
apparition_realisateur = 8
apparition_acteur = 10
apparition_production = 5
len_production = 3

ACTEURS ET REALISATEURS

In [34]:
liste_films = pd.DataFrame(df_films['film_id_out_KNN'])
liste_films = liste_films.drop_duplicates()

In [35]:
df_title_principals = pd.read_csv('BD/title.principals.tsv.gz', compression = 'gzip', sep = '\t')

In [36]:
df_acteurs = df_title_principals[(df_title_principals['category'] == 'actor') | (df_title_principals['category'] == 'actress') | (df_title_principals['category'] == 'self')]

df_realisateurs = df_title_principals[df_title_principals['category'] == 'director']

In [37]:
df_title_principals = 0

In [38]:
df_title_crew = pd.read_csv('BD/title.crew.tsv.gz', compression = 'gzip', sep = '\t')

In [39]:
df_title_crew = df_title_crew.drop('writers', axis = 1)

df_title_crew = pd.merge(liste_films,
                         df_title_crew,
                         left_on = 'film_id_out_KNN',
                         right_on = 'tconst',
                         how = 'left')

df_title_crew = df_title_crew.drop('film_id_out_KNN', axis = 1)

df_title_crew = df_title_crew.rename({'directors': 'nconst'}, axis = 1)

df_title_crew['nconst'] = df_title_crew['nconst'].astype(str)

df_title_crew['nconst'] = df_title_crew['nconst'].apply(lambda x : x.split(","))

df_title_crew = df_title_crew.explode('nconst')

df_title_crew = df_title_crew[~(df_title_crew['nconst'] == " ")]

df_title_crew['tconst'] = df_title_crew['tconst'] + ', '

df_title_crew = df_title_crew.groupby('nconst').sum()

df_title_crew = df_title_crew.reset_index()

In [40]:
df_name_basics = pd.read_csv('BD/name.basics.tsv.gz', compression = 'gzip', sep = '\t')

In [41]:
df_name_basics = df_name_basics.drop(['birthYear', 'deathYear', 'primaryProfession'], axis = 1)

df_name_basics = df_name_basics.rename({'knownForTitles' : 'tconst'}, axis = 1)

df_name_basics['tconst'] = df_name_basics['tconst'].apply(lambda x : x.split(','))

df_name_basics['nconst'] = df_name_basics['nconst'].astype(str)

df_name_basics['nconst'] = df_name_basics['nconst'].apply(lambda x : x.replace(' ', ''))

df_acteurs = name(liste_films, df_acteurs)

df_realisateurs = name(liste_films, df_realisateurs)

In [42]:
df_acteurs_name = pd.merge(df_acteurs,
                            df_name_basics,
                            on = 'nconst',
                            how = 'left')

In [43]:
df_acteurs = 0
df_acteurs_name = acteur_name(df_acteurs_name)
df_realisateurs_name = realisateur_name(df_realisateurs)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tconst'] = df['tconst'].apply(lambda x : x.replace(' ', '').replace('[', '').replace(']', '').replace("'", ''))
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tconst'] = df['tconst'].apply(lambda x : x.split(','))


In [44]:
df_acteurs, df_realisateurs, df_name_basics, df_title_crew = 0, 0, 0, 0

In [45]:
df_realisateurs_name.to_csv('P2_G5_realisateurs.csv.gz', index=False, compression='gzip')
df_realisateurs = df_realisateurs_name

df_acteurs_name.to_csv('P2_G5_acteurs.csv.gz', index=False, compression='gzip')
df_acteurs = df_acteurs_name

In [46]:
# DF ACTEURS
 
df_acteurs = df_acteurs.rename({'knownForTitles' : 'films'}, axis = 1)

df_acteurs2 = df_acteurs.explode('films')

acteurs_repetitions = pd.DataFrame(df_acteurs2['acteur'].value_counts()).reset_index()

acteurs_repetitions = acteurs_repetitions[acteurs_repetitions['count'] > apparition_acteur]

acteurs_recurrents = pd.merge(acteurs_repetitions,
                              df_acteurs,
                              how = 'left',
                              on = 'acteur')

acteurs_recurrents2 = acteurs_recurrents.explode('films')

acteurs_recurrents2['films'] = acteurs_recurrents2['films'].str.replace("'","")
acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'] + ', '

acteurs_recurrents2 = acteurs_recurrents2.groupby('films').sum()

acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'].astype(str)

acteurs_recurrents2['acteur'] = acteurs_recurrents2['acteur'].apply(lambda x : x.replace(", , ", ''))

acteurs_recurrents2 = acteurs_recurrents2.reset_index()
acteurs_recurrents2 = acteurs_recurrents2[['films', 'acteur']]

df_films = pd.merge(df_films,
                    acteurs_recurrents2,
                    how = 'left',
                    left_on = 'film_id_out_KNN',
                    right_on = 'films')

acteurs_repetitions['acteur'] = acteurs_repetitions['acteur'].astype(str)

In [47]:
# DF REALISATEURS

realisateurs_repetitions = pd.DataFrame(df_realisateurs['realisateurs'].value_counts()).reset_index()

realisateurs_repetitions = realisateurs_repetitions[realisateurs_repetitions['count'] > apparition_realisateur]

realisateurs_repetitions2 = realisateurs_repetitions.groupby('realisateurs').sum()

realisateurs_repetitions2 = realisateurs_repetitions2.reset_index()

realisateurs_connus = pd.merge(realisateurs_repetitions2,
                               df_realisateurs,
                               on = 'realisateurs',
                               how = 'left')

realisateurs_connus = realisateurs_connus[['knownForTitles', 'realisateurs']]

realisateurs_connus['realisateurs'] = realisateurs_connus['realisateurs'] + ','

realisateurs_connus2 = realisateurs_connus.groupby('knownForTitles').sum()

df_films = pd.merge(df_films,
                    realisateurs_connus2,
                    how = 'left',
                    left_on = 'film_id_out_KNN',
                    right_on = 'knownForTitles')

df_realisateurs['realisateurs'] = df_realisateurs['realisateurs'].astype(str)

df_films['realisateurs'] = df_films['realisateurs'].astype(str)

df_films = df_films.rename({'acteur' : 'acteur_out_KNN',
                            'realisateurs' : 'realisateurs_out_KNN'}, axis = 1)

In [49]:
df_realisateurs_name = pd.read_csv('BD/P2_G5_realisateurs.csv.gz', compression = 'gzip')

In [50]:
df_acteurs_name = pd.read_csv('BD/P2_G5_acteurs.csv.gz', compression = 'gzip')

In [51]:
df_realisateurs_name['knownForTitles'] = df_realisateurs_name['knownForTitles'] + ','
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'] + ','

df_realisateurs_name = df_realisateurs_name.groupby('realisateurs').sum()

df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'].apply(lambda x : x.split(','))
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'].apply(lambda x : set(x))
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'].apply(lambda x : str(x))
df_realisateurs_name['primaryName'] = df_realisateurs_name['primaryName'].apply(lambda x : x.replace('{', '').replace('}', '').replace(',', '').replace('"', "").replace("[", '').replace("]", '').replace("'", ''))

df_realisateurs_name['knownForTitles'] = df_realisateurs_name['knownForTitles'].apply(lambda x : x.split(','))
df_realisateurs_name['knownForTitles'] = df_realisateurs_name['knownForTitles'].apply(lambda x : set(x))
df_realisateurs_name = df_realisateurs_name.reset_index()

In [52]:
dico_acteurs = {}

for n in range(len(df_acteurs_name)):
    dico_acteurs.update({df_acteurs_name['acteur'].iloc[n] : df_acteurs_name['primaryName'].iloc[n]})

dico_realisateurs = {}

for n in range(len(df_realisateurs_name)):
    dico_realisateurs.update({df_realisateurs_name['realisateurs'].iloc[n] : df_realisateurs_name['primaryName'].iloc[n][1:]})

dico_total = dico_acteurs | dico_realisateurs

dico_total_final = {}

for element in dico_total:
    if element not in dico_total_final.keys():
        dico_total_final.update({element : dico_total[element]})

In [53]:
list_colonnes(df_films,'realisateurs_out_KNN')

In [54]:
liste_totale_realisateurs, liste_limitee_realisateurs = liste_recurrence(df_films, 'realisateurs_out_KNN')

In [55]:
# classement_element(df_films, 'realisateurs_out_KNN', liste_totale_realisateurs, dico_total_final)

In [56]:
bool_colonnes(df_films, liste_limitee_realisateurs, 'realisateurs_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceW

ACTEURS

In [57]:
list_colonnes(df_films,'acteur_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:275: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())


In [58]:
liste_totale_acteur, liste_limitee_acteur = liste_recurrence(df_films, 'acteur_out_KNN')

In [59]:
# classement_element(df_films, 'acteur_out_KNN', liste_totale_acteur, dico_total_final)

In [60]:
bool_colonnes(df_films, liste_limitee_acteur, 'acteur_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceW

GENRE

In [61]:
list_colonnes(df_films,'genre_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:275: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())


In [62]:
liste_totale_genre, liste_limitee_genre = liste_recurrence(df_films, 'genre_out_KNN')

In [63]:
# classement_element(df_films, 'genre_out_KNN', liste_totale_genre, dico_total_final)

In [64]:
bool_colonnes(df_films, liste_limitee_genre, 'genre_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceW

PRODUCTION COMPANY

In [65]:
list_colonnes(df_films,'production_companies_name_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:275: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne}2'] = df_films[colonne].apply(lambda x: x.replace('"', "").replace("[", '').replace("]", '').replace("'", '').replace(" ", '').lower())


In [66]:
liste_totale_production, liste_limitee_production = liste_recurrence(df_films, 'production_companies_name_out_KNN')

In [67]:
# classement_element(df_films, 'production_companies_name_out_KNN', liste_totale_production, dico_total_final)

In [68]:
bool_colonnes(df_films, liste_limitee_production, 'production_companies_name_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceW

In [69]:
df_films['production_companies_name_Marvel'] = df_films['production_companies_name_out_KNN'].apply(lambda x: 'marvel' in x)
df_films['production_companies_name_Disney'] = df_films['production_companies_name_out_KNN'].apply(lambda x: 'disney' in x)

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3847665447.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films['production_companies_name_Marvel'] = df_films['production_companies_name_out_KNN'].apply(lambda x: 'marvel' in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3847665447.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films['production_companies_name_Disney'] = df_films['production_companies_name_out_KNN'].apply(lambda x: 'disney' in x)


TITLE

In [ ]:
df_films['title_out_KNN'] = ''

df_films['title_out_KNN'] = df_films.apply(lambda x : titre(x['film_id_tmbd']), axis = 1)

DEBUT DE TITRE

In [70]:
liste_limitee_title = colonne_debut_titre(df_films, 'title_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:250: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'debut{colonne}'] = df_films[colonne].apply(debut2)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:252: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'debut_critere_{colonne}'] = ''


In [71]:
bool_colonnes(df_films, liste_limitee_title, 'debut_critere_title_out_KNN')

/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_films[f'{colonne[:-8]}_{genre}'] = df_films[f'{colonne}'].apply(lambda x: genre in x)
/var/folders/2b/18xblx_n4lv24wz932vjckrm0000gp/T/ipykernel_20297/3642231367.py:216: PerformanceW

BOOL DES COLONNES

In [72]:
df_films = transfo_col_bool(df_films)

In [73]:
colonnes = list(df_films.select_dtypes(include = 'number').columns)
supprimer = []

df_null = pd.DataFrame(df_films[colonnes].sum()).transpose()

for element in colonnes:
    if df_null[element].iloc[0] == 0:
        supprimer.append(element)

if len(supprimer) > 0:
    df_films = df_films.drop(supprimer, axis = 1)

FILLNA

In [74]:
df_films_str = df_films.select_dtypes(exclude = 'number')
df_films_num = df_films.select_dtypes(include = 'number')
df_films_str = df_films_str.fillna('Unknown')
df_films_num = df_films_num.fillna(0)

In [ ]:
df_films = pd.concat([df_films_str, df_films_num], axis = 1)

EXPORT

In [75]:
df_films = df_films.drop(['films'], axis = 1)

In [76]:
len(df_films), len(df_films.columns)

(86663, 582)

In [77]:
df_films.to_csv('P2_G5_films.csv.gz', index=False, compression='gzip')